### Data Crawler ###

In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import time, ctime

labels = {
    "政治" : 0, # politics
    "運動" : 1, # sport
    "財經" : 2, # finance
    "社會" : 3, # society
    "房產" : 4, # house2
    "國際" : 5, # global
    "娛樂" : 6, # entertainment
}

url = "https://www.nownews.com/cat/entertainment/page/"
label = 6

In [62]:
news_df = pd.DataFrame([])

for p in range(1,101):
    url2 = url + str(p)
    header = {
        "accept-language":"zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
        "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
    }
    response = requests.get(url2, headers=header)
    
    html = BeautifulSoup(response.text)
    div = html.find("div", class_="td_block_inner tdb-block-inner td-fix-index")
    news = div.find_all("div", class_="td-module-meta-info")

    for n in news:
        n_df = pd.DataFrame([])

        title_h3 = n.find("h3", class_="entry-title td-module-title")
        title = title_h3.text.strip()
        link = title_h3.find("a")["href"].strip()
        date = n.find("div", class_="td-editor-date").text.strip()
        content = n.find("div", class_="td-excerpt").text
    #     content = content.split("\n")[0].strip() # 只取文章第一段

        dic = {
            'title' : title,
            'date' : date,
            'content' :content,
            'link' : link
        }

        news_df = news_df.append(pd.DataFrame(dic,index=[0]))
print("length = ", len(news_df))

length =  1200


### Data Processing###

In [4]:
import re
import jieba
import jieba.analyse

# [\u4e00-\u9fa5] 取中文
pattern = re.compile(r'(\d+)') # 取數字
pattern2 = re.compile(r'[\d+\u4e00-\u9fa5]+') # 取中文加數字

# 全文分詞
# txt = news_df['content'].values
# for t in txt:
#     txt_list = pattern2.findall(t)
#     txt2 = ''.join(txt_list)
#     print(txt2)
    
#     seg_list = jieba.cut(txt2, cut_all=False) # 全模式
#     print("Full Mode: " + "/ ".join(seg_list)) 
# -------------------------------------------------------------------------

# 關鍵字提取(基于TF-IDF算法)
# topK: 返回前幾個權重大的關鍵字，默認為20
# withWeight: 是否返回關鍵字權重，默認為False
# allowPOS: 包括指定詞性的詞，默認為空，即不篩選
def getTxtArray(txt, K, word_index, new_word_index):
    txt_list = pattern2.findall(txt)
    txt2 = ''.join(txt_list)

    keywords = jieba.analyse.extract_tags(sentence=txt2, topK=K, withWeight=True, allowPOS=('n','nr','ns'))
    # 基于TextRank算法的关键词抽取
#     keywords2 = jieba.analyse.textrank(txt2, topK=20, withWeight=True, allowPOS=('n','nr','ns'))
#     print('length = ', len(keywords))

    words = []
    for item in keywords:
#         print(item[0],item[1])
        key = item[0]
        words.append(key)
        if key in word_index.keys():
            continue     
        else:
            if len(word_index) == 0:
                word_index[key] = 1
                new_word_index[key] = 1
            else:
                last_index = word_index[max(word_index, key=word_index.get)]
                word_index[key] = int(last_index) + 1
                new_word_index[key] = int(last_index) + 1   
#     print(words)
#     print([word_index[w] for w in words])
    return [word_index[w] for w in words]

### BigQuery ###

In [5]:
from google.cloud import bigquery
from google.oauth2 import service_account
from googleapiclient import discovery
from pythonbq import pythonbq
import uuid

my_project_id='maximal-cabinet-254805'
my_dataset_id='news_keywords'
credentials_path = 'C:/Users/GameToGo/MyCredentials/My First Project-7226ff9d97d3.json'

def createTable(service, table_id, schema):
    # first checks if table already exists.
    r = service.tables().list(projectId=my_project_id,
                              datasetId=my_dataset_id).execute()
    table_exists = [row['tableReference']['tableId'] for row in
                    r['tables'] if row['tableReference']['tableId'] == my_table_id]
#     table_exists = False # 第一次無 r['tables']
    
    if not table_exists:
        body = {
                'tableReference': {
                        'tableId': table_id,
                        'projectId': my_project_id,
                        'datasetId': my_dataset_id
                },
                'schema': schema
        }
        service.tables().insert(projectId=my_project_id,
                                datasetId=my_dataset_id,
                                body=body).execute()
        print('table create succeed!')
    else:
        print('table already exists!')

def appendKeywordRows(word_index):
    rows = []
    for w in word_index:
        j = {}
        jj = {}

        j['id'] = word_index[w]
        j['keyword'] = w

        jj['json'] = j
        jj['insertId'] = str(uuid.uuid4())

        rows.append(jj)
    return rows

def appendNewsRows(news_df):
    rows = []
    for n in news_df.values:
        j = {}
        jj = {}

        j['zdate'] = n[0]
        j['title_array'] = str(n[1])
        j['content_array'] = str(n[2])
        j['web_link'] = n[3]        
        j['label'] = n[4]
        
        jj['json'] = j
        jj['insertId'] = str(uuid.uuid4())

        rows.append(jj)
    return rows

def insertData(service, rows, table_id):
    body = {
        'rows': rows
    }
    service.tabledata().insertAll(projectId=my_project_id,
                                    datasetId=my_dataset_id,
                                    tableId=table_id,
                                    body=body).execute(num_retries=5)
    print('data insert succeed!')
    
def getWord_index():
    myProject=pythonbq(
      bq_key_path = credentials_path,
      project_id = my_project_id
    )
    SQL_CODE="""
        SELECT * FROM `maximal-cabinet-254805.news_keywords.keywords` 
        order by id
    """

    output=myProject.query(sql=SQL_CODE)

    word_index = {}
    for o in output.values:
        word_index[o[1]] = o[0]
        
    return word_index

### Main Flow ###

In [63]:
start = time()

# word_index = getWord_index() # old index
new_word_index = {} # new index

title_df = news_df['title'].apply(lambda x: getTxtArray(x, 5, word_index, new_word_index))
content_df = news_df['content'].apply(lambda x: getTxtArray(x, 20, word_index, new_word_index))

news_df = pd.DataFrame({
                'zdate':news_df['date'].values, 
                'title_array':title_df.values, 
                'content_array':content_df.values,
                'link':news_df['link'].values
            })
news_df['label'] = label

print("word_index = ", len(word_index))
print("new_word_index = ", len(new_word_index))
print("news_df = ", len(news_df))
print(f'training data loaded for {(time()-start)/60:.2f} mins')

word_index =  30842
new_word_index =  5058
news_df =  1200
training data loaded for 34.48 secs


In [64]:
credentials = service_account.Credentials.from_service_account_file(
    credentials_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

service = discovery.build('bigquery', 'v2', credentials = credentials)

# -------------------------------------------------------------------------
# Table建立

# keywords
# schema = {u'fields': [
#         {u'type': u'INTEGER', u'name': u'id', u'mode': u'REQUIRED'}, 
#         {u'type': u'STRING', u'name': u'keyword', u'mode': u'REQUIRED'}
# ]}

# news
# schema = {u'fields': [
#         {u'type': u'STRING', u'name': u'zdate', u'mode': u'REQUIRED'},
#         {u'type': u'STRING', u'name': u'title_array', u'mode': u'REQUIRED'}, 
#         {u'type': u'STRING', u'name': u'content_array', u'mode': u'REQUIRED'},
#         {u'type': u'STRING', u'name': u'web_link', u'mode': u'NULLABLE'},
#         {u'type': u'INTEGER', u'name': u'label', u'mode': u'REQUIRED'}
# ]}

# createTable(service, 'keywords', schema) # 建立table
# createTable(service, 'news', schema) # 建立table

# -------------------------------------------------------------------------
# 匯入資料集

Keyword_rows = appendKeywordRows(new_word_index) # 產生資料列
insertData(service, Keyword_rows, 'keywords') # 匯入資料

News_rows = appendNewsRows(news_df) # 產生資料列
insertData(service, News_rows, 'news') # 匯入資料


data insert succeed!
data insert succeed!


In [245]:
rows

[{'json': {'zdate': '2020-02-03',
   'title_array': '[1, 2, 3, 4, 5]',
   'content_array': '[49, 50, 51, 52, 53, 1, 54, 55, 56, 57, 58, 36, 59, 60, 61, 62, 63, 64, 65, 66]',
   'web_link': 'https://www.nownews.com/news/20200203/3914296/',
   'label': 1},
  'insertId': 'dae1de66-c556-42a5-ab8c-56c5d2df3058'},
 {'json': {'zdate': '2020-02-03',
   'title_array': '[6, 7, 8, 9, 10]',
   'content_array': '[67, 7, 8, 10, 68, 6, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]',
   'web_link': 'https://www.nownews.com/news/20200203/3914438/',
   'label': 1},
  'insertId': '40a33e00-0f04-42e3-9834-718ef1e2bc8c'},
 {'json': {'zdate': '2020-02-03',
   'title_array': '[11, 12, 13, 14, 15]',
   'content_array': '[83, 14, 15, 11, 84, 85, 86, 87, 88, 89, 12, 90, 91, 92, 93, 94, 95, 96, 97, 98]',
   'web_link': 'https://www.nownews.com/news/20200203/3914333/',
   'label': 1},
  'insertId': 'ca3e5038-4e86-4dc7-9277-f6660094b462'},
 {'json': {'zdate': '2020-02-03',
   'title_array': '[16, 17, 18,

### BigQuery Search###

In [166]:
credentials = service_account.Credentials.from_service_account_file(
    "C:/Users/GameToGo/MyCredentials/My First Project-7226ff9d97d3.json",
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

query_job = client.query(
    "SELECT * FROM `maximal-cabinet-254805.news_keywords.keywords` order by id "
)

for row in query_job: 
    # Row values can be accessed by field name or index
#     assert row[0] == row.name == row["name"]
    print(row)

Row((0, '球員'), {'id': 0, 'keyword': 1})
Row((1, '史密斯'), {'id': 0, 'keyword': 1})
Row((2, '歐尼爾'), {'id': 0, 'keyword': 1})
Row((3, '球星'), {'id': 0, 'keyword': 1})
Row((4, '短片'), {'id': 0, 'keyword': 1})
Row((5, '傳奇'), {'id': 0, 'keyword': 1})
Row((6, '墜機'), {'id': 0, 'keyword': 1})
Row((7, '前隊友'), {'id': 0, 'keyword': 1})
Row((8, '比威爾'), {'id': 0, 'keyword': 1})
Row((9, '球場'), {'id': 0, 'keyword': 1})
Row((10, '腳步'), {'id': 0, 'keyword': 1})
Row((11, '專輯'), {'id': 0, 'keyword': 1})
Row((12, '籃球'), {'id': 0, 'keyword': 1})
Row((13, '奧斯卡'), {'id': 0, 'keyword': 1})
Row((14, '動畫'), {'id': 0, 'keyword': 1})
Row((15, '趣事'), {'id': 0, 'keyword': 1})
Row((16, '影星'), {'id': 0, 'keyword': 1})
Row((17, '球迷'), {'id': 0, 'keyword': 1})
Row((18, '世界'), {'id': 0, 'keyword': 1})


In [173]:
from pythonbq import pythonbq

myProject=pythonbq(
  bq_key_path="C:/Users/GameToGo/MyCredentials/My First Project-7226ff9d97d3.json",
  project_id=my_project_id
)
SQL_CODE="""
    SELECT * FROM `maximal-cabinet-254805.news_keywords.keywords` 
    order by id
"""

output=myProject.query(sql=SQL_CODE)
print(type(output))
output

Downloading: 100%|██████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 135.23rows/s]


<class 'pandas.core.frame.DataFrame'>


,id,keyword
0,0,球員
1,1,史密斯
2,2,歐尼爾
3,3,球星
4,4,短片
5,5,傳奇
6,6,墜機
7,7,前隊友
8,8,比威爾
9,9,球場
